In [1]:
import requests
import yaml
import re
from tqdm import tqdm
from time import sleep
import time
import os
os.chdir("C:\\Users\weejw\Documents\steamRecommendation")
from db_config.db_funtion import db_execute


In [2]:
resp = requests.get(" http://api.steampowered.com/ISteamApps/GetAppList/v0002/?format=json")
app_list = resp.json()
apps = app_list['applist']['apps']

with open('C:\\Users\weejw\Documents\steamRecommendation\db_config\db_auth.yaml', 'r') as file:
    db_auth = yaml.safe_load(file)


In [3]:
len(apps)


131984

In [4]:
def data_insert(game_info):
    need_data = ['name','steam_appid', 'required_age', 'is_free', 'short_description', 'developers', 'publishers', 'price_overview', 
             'platforms', 'release_date', 'supported_languages', 'categories', 'genres', 'recommendations']
    
    need_data_dic ={}
    print(game_info)
    for key, data in game_info.items():
        if key in need_data:
            if key in ["developers", "publishers"]:
                data = data[0]

            elif key == "price_overview":
                need_data_dic['final_price'] = data['final']
                need_data_dic['initial_price'] = data['initial']
                need_data_dic['discount_percent'] = data['discount_percent']
                continue

            elif key == "platforms":
                res = ""
                for platform, boolean in data.items():
                    if boolean:
                       res += platform 
                data = res

            elif key == "release_date":
                if data['coming_soon']:
                    data = 'coming_soon'
                else:
                    data = data['date']
            elif key == "supported_languages":
                data = re.sub('<.*?>|[-*]', '',data).replace("음성이 지원되는 언어",", 음성이 지원되는 언어")

            elif key == "categories":
                category_id = []
                category_value = []
                for category_info in data:
                    category_id.append(str(category_info['id']))
                    category_value.append(category_info['description'])
                need_data_dic['categories_id'] = ','.join(category_id)
                need_data_dic['categories'] = ','.join(category_value)
                continue

            elif key == "genres":
                genre_id = []
                genre_value = []
                for genre_info in data:
                    genre_id.append(str(genre_info['id']))
                    genre_value.append(genre_info['description'])
                need_data_dic['genres_id'] = ','.join(genre_id)
                need_data_dic['genres'] = ','.join(genre_value)
                continue

            elif key == "recommendations":
                data = data['total']

            if type(data) is str:
                need_data_dic[key] = re.sub('(<([^>^-]+)>)', '',data)
            else:
                need_data_dic[key] = data
                
    placeholders = ', '.join(['%s'] * len(need_data_dic))
    columns = ', '.join(need_data_dic.keys())
    
    sql = "INSERT IGNORE INTO %s ( %s ) VALUES ( %s )" % ('game_info', columns, placeholders)
    return db_function.execute_insert(sql, list(need_data_dic.values()))

In [19]:
start_time = time.time()
try_cnt = 0
db_function = None
try:    
    db_function = db_execute(db_auth['local_db']["host"], 
                             db_auth['local_db']["port"],
                             db_auth['local_db']["user"],
                             db_auth['local_db']["db"], 
                             db_auth['local_db']["passwd"])
    for app_info in tqdm(apps):
        try_cnt +=1
        current_time = time.time()
        elapsed_time = current_time - start_time

        if try_cnt > 200 and elapsed_time < 300:
            sleep(60)
            try_cnt = 0
            start_time = time.time()

        try:
            appid = app_info['appid']
            sql = "SELECT id FROM game_info WHERE steam_appid=%s" % appid
            if db_function.execute_select(sql):
                continue

            url = f"https://store.steampowered.com/api/appdetails?appids={appid}&l=korean"

            resp = requests.get(url)
            if not resp.ok:
                print(url, resp)

            game_info_resp = resp.json()
            game_info = list(game_info_resp.values())[0]
            if game_info_resp and game_info['success'] and game_info['data']['type'] == "game":
                    res = data_insert(game_info['data'])
            sleep(5)

        except KeyboardInterrupt as e:
            break

        except:
            raise
            
finally:
    db_function.curs.close()
    db_function.conn.close()

  0%|                                                                                                                                                                                                                                                    | 0/131984 [00:00<?, ?it/s]

{'216938': {'success': False}}


  0%|                                                                                                                                                                                                                                        | 1/131984 [00:05<192:14:44,  5.24s/it]

{'660010': {'success': False}}


  0%|                                                                                                                                                                                                                                        | 2/131984 [00:10<194:45:25,  5.31s/it]

...

  0%|                                                                                                                                                                                                                                       | 62/131984 [02:32<172:59:59,  4.72s/it]

{'394390': {'success': True, 'data': {'type': 'game', 'name': 'Daydreamer: Awakened Edition', 'steam_appid': 394390, 'required_age': 0, 'is_free': False, 'controller_support': 'full', 'detailed_description': 'Daydreamer is a platformer based in a beautifully-illustrated Post-Apocalyptic World from the mind of artist Roland Womack. As the last human survivor, you are on a quest to rescue the Immortal Gatekeeper, the last hope for a dying world. <br>The game focuses on high-intensity Boss Battles in a classic 90s Arcade fashion with a modern twist that incorporates support characters to aid you in battle. Fight your way through 14 unique bosses across a set of imaginative dreamscapes, employing a number of weapons and abilities to overcome your foes and the environment.<br><br><strong>[Best played with a Gamepad, unless you are keanu reeves with a keyboard]</strong>', 'about_the_game': 'Daydreamer is a platformer based in a beautifully-illustrated Post-Apocalyptic World from the mind of 

  0%|                                                                                                                                                                                                                                       | 63/131984 [02:38<178:26:05,  4.87s/it]

{'394400': {'success': True, 'data': {'type': 'dlc', 'name': 'Three Digits - Soundtrack', 'steam_appid': 394400, 'required_age': 0, 'is_free': False, 'detailed_description': 'The soundtrack for Three Digits. Includes three calm, soothing songs.<br />\r\n<br />\r\nTracklist:<br />\r\nSong 1<br />\r\nSong 2<br />\r\nSong 3', 'about_the_game': 'The soundtrack for Three Digits. Includes three calm, soothing songs.<br />\r\n<br />\r\nTracklist:<br />\r\nSong 1<br />\r\nSong 2<br />\r\nSong 3', 'short_description': 'The soundtrack for Three Digits. Includes three calm, soothing songs in high quality .wav format.', 'fullgame': {'appid': '390570', 'name': 'Three Digits'}, 'supported_languages': '영어<strong>*</strong><br><strong>*</strong>음성이 지원되는 언어', 'header_image': 'https://cdn.akamai.steamstatic.com/steam/apps/394400/header.jpg?t=1447377536', 'website': 'http://www.cleverweek.com', 'pc_requirements': {'minimum': '<strong>최소:</strong><br><ul class="bb_ul"><li><strong>운영체제:</strong> Any</li></

  0%|                                                                                                                                                                                                                                        | 63/131984 [02:43<95:07:05,  2.60s/it]
